In [51]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

In [52]:
from time import sleep
import random
import pandas as pd

In [53]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [54]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import requests

In [55]:
# 바이너리 오류가 발생할 경우
# 크롬 바이너리 경로와 크롬 드라이버 바이너리 경로를 둘다 명시해줘야함
options = webdriver.ChromeOptions()
options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
chrome_driver_binary = "./chromedriver"
driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
print("##########driver open##########")

/var/folders/rg/mxvp06m56_g09wl1jz84qkhh0000gn/T/ipykernel_17337/699548518.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
/var/folders/rg/mxvp06m56_g09wl1jz84qkhh0000gn/T/ipykernel_17337/699548518.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)


##########driver open##########


In [56]:
wait = WebDriverWait(driver, 600)

# 크롤링

## url 접속 및 df 열 정의

In [57]:
site = "http://ie.hanyang.ac.kr/index.php?pg=1&page=list&hCode=BOARD&bo_idx=1&sfl=&stx="

driver.get(url = site)
sleep(random.randint(1, 3))

data = {
        '번호': [],
        '분류': [],
        '등록시간': [],
        '제목': [],
        '본문': [],
        'url': [],
       }

## 공지 수 
## 한 페이지의 최대 콘텐츠 수
## 총 콘텐츠 수

In [58]:
# 페이지의 콘텐츠 번호 리스트
content_num = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/table/tbody/tr/td[1]")
print("첫 페이지의 콘텐츠 수 :", len(content_num))

# 공지의 수
announce_num = 0
for i in range(len(content_num)):
    if content_num[i].text == "공지":
        announce_num += 1
    else:
        break
print("공지의 개수 :", announce_num)

max_content_num = len(content_num)-announce_num
print("한 페이지의 공지를 제외한 최대 콘텐츠의 개수:", max_content_num)

# 총 콘텐츠의 수
num_of_content = content_num[announce_num].text
print("공지를 제외한 총 콘텐츠의 개수 :", num_of_content)

첫 페이지의 콘텐츠 수 : 41
공지의 개수 : 21
한 페이지의 공지를 제외한 최대 콘텐츠의 개수: 20
공지를 제외한 총 콘텐츠의 개수 : 738


## 공지 크롤링

- 콘텐츠 크롤링과 함께 묶지 않은 이유 : 그렇게 할 경우 매 페이지마다 공지를 중복으로 크롤링함

In [60]:
for i in range(len(content_num)):
    
    # 페이지 로드 대기
    sleep(random.randint(1, 3))
    
    # stale 오류 때문에 위의 변수를 계속 사용하지 못하고 계속 업데이트 해야함
    content_num = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/table/tbody/tr/td[1]")
    
    if content_num[i].text == "공지":
        print("--------------------------------------------")
        
        data['번호'].append("공지")
        print('번호 :', "공지")
        data['분류'].append("산업공학과")
        print('분류 :', "산업공학과")
        
        # 글 클릭
        con_button = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/table/tbody/tr[" + str(i+1) + "]/td[2]/a")[0]
        con_button.click()
        
        # 등록시간 수집
        upload_time = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[2]/td/ul/li[5]")[0]
        upload_time = upload_time.text
        data['등록시간'].append(upload_time)
        print('등록시간 :', upload_time)
        
        # 제목 수집
        title = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[1]/th")[0]
        title = title.text
        data['제목'].append(title)
        print('제목 :', title)

        # 본문 수집
        main = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/tbody/tr/td")[0]
        main = main.get_attribute('innerHTML')
        data['본문'].append(main)
        print('본문 :', main[:20], '~')
        
        # url 수집
        data['url'].append(driver.current_url)
        print('url :',driver.current_url)
        
        # 뒤로가기
        driver.back()
    else:
        break
        

--------------------------------------------
번호 : 공지
학과 : 산업공학과
등록시간 : 2023-08-03 13:03:52
제목 : [학부]음악대학 마이크로전공 신설 및 모집 안내
본문 : ['']
url : http://ie.hanyang.ac.kr/index.php?page=view&pg=1&idx=2044&hCode=BOARD&bo_idx=1&sfl=&stx=
--------------------------------------------
번호 : 공지
학과 : 산업공학과
등록시간 : 2023-07-26 12:20:14
제목 : [학부]2023학년도 2학기 군 e-러닝 과정 안내
본문 : ['* 한양대학교(서울/ERICA) 군 e-러닝 과정 주요 내용', '  ', '구분', '내용', '대상', '재학 중 휴학계 제출 후 입대한 학생(군 복무자)', '   - 국방부 소속 병사(현역 병사, 상근예비역)', '   - 병무청 소속 사회복무요원', '※비고 : 2023년 10월 4일 이전 제대자 및 복학신청자는 수강불가', '학점인정 범위 및 기준', '한 학기당 최대 6학점, 군 복무중 최대 9학점 이수가능', '수강기간', '정규학기(1학기, 2학기)', '수강신청 및 수강료 납부', '나라사랑 포털(www.narasarang.or.kr)로 학습자 직접 신청 및 납부', '※2학기 수강신청 기간 : 2023.08.21.(월) 18:00 ~ 2023.09.24.(일)', '수강 LMS', 'm.selc.or.kr', '비고', '소속기관(국방부, 병무청)에서 수강료 80% + 수수료 11,000원 지원(문의 : 나라사랑포털 1522-0770)', ' ', '구분', '내용', '대상', '● 재학 중 휴학계 제출 후 입대한 학생(군 복무자)', '   - 국방부 소속 병사(현역 병사, 상근예비역)', '   - 병무청 소속 사회복무요원', '※비고 : 2023년 10월 4일 이전 제대자 및 복학신청자는 수강불가', 

KeyboardInterrupt: 

## 공지 제외 콘텐츠 크롤링

In [49]:
for i in range((int(num_of_content)//max_content_num)+1):
    
    # 마지막 페이지의 콘텐츠 수가 다를 수 있으므로
    content_num = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/table/tbody/tr/td[1]")
    
    for i in range(len(content_num)):

        # 페이지 로드 대기
        sleep(random.randint(1, 3))

        # stale 오류 때문에 위의 변수를 계속 사용하지 못하고 계속 업데이트 해야함
        content_num = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/table/tbody/tr/td[1]")

        if content_num[i].text != "공지":
            print("--------------------------------------------")
            
            data['번호'].append(content_num[i].text)
            print('번호 :', content_num[i].text)
            data['분류'].append("산업공학과")
            print('분류 :', "산업공학과")

            # 글 클릭
            con_button = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/table/tbody/tr[" + str(i+1) + "]/td[2]/a")[0]
            con_button.click()

            # 등록시간 수집
            upload_time = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[2]/td/ul/li[5]")[0]
            upload_time = upload_time.text
            data['등록시간'].append(upload_time)
            print('등록시간 :', upload_time)

            # 제목 수집
            title = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/thead/tr[1]/th")[0]
            title = title.text
            data['제목'].append(title)
            print('제목 :', title)

            # 본문 수집
            main = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/table/tbody/tr/td")[0]
            main = main.get_attribute('innerHTML')
            data['본문'].append(main)
            print('본문 :', main[:20], '~')

            # url 수집
            data['url'].append(driver.current_url)
            print('url :',driver.current_url)

            # 뒤로가기
            driver.back()
        else:
            continue
    next_button = driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/form[2]/div/ul/a[11]/img")[0]
    next_button.click()


--------------------------------------------
번호 : 738
학과 : 산업공학과
등록시간 : 2023-07-24 12:42:15
제목 : [학부]2022학년도 후기 학위수여식 공로상 대상자 신청 안내
본문 : []
url : http://ie.hanyang.ac.kr/index.php?page=view&pg=1&idx=2020&hCode=BOARD&bo_idx=1&sfl=&stx=
--------------------------------------------
번호 : 737
학과 : 산업공학과
등록시간 : 2023-06-22 06:52:43
제목 : 2023년 8월 졸업예정자의「학사학위취득유예」및「다전공포기」신청 안내
본문 : []
url : http://ie.hanyang.ac.kr/index.php?page=view&pg=1&idx=1993&hCode=BOARD&bo_idx=1&sfl=&stx=
--------------------------------------------
번호 : 736
학과 : 산업공학과
등록시간 : 2023-06-15 11:47:47
제목 : 2023학년도 2학기 학생생활관 정규입사 모집일정 안내
본문 : []
url : http://ie.hanyang.ac.kr/index.php?page=view&pg=1&idx=1987&hCode=BOARD&bo_idx=1&sfl=&stx=
--------------------------------------------
번호 : 735
학과 : 산업공학과
등록시간 : 2023-06-13 10:52:35
제목 : 2023학년도 IC-PBL글로벌 프론티어 모집 안내
본문 : []
url : http://ie.hanyang.ac.kr/index.php?page=view&pg=1&idx=1981&hCode=BOARD&bo_idx=1&sfl=&stx=
--------------------------------------------
번호 : 734
학과 : 산업공학과
등

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(data)

df.head(8)

In [ ]:
# driver.close()

In [ ]:
# df = pd.DataFrame(data)

# print(df)

In [ ]:
# df.to_csv('output_350to369.csv', index=False)